# ![iceberg-logo](https://www.apache.org/logos/res/iceberg/iceberg.png)

#### Create Spark Session 

In [14]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Jupyter").getOrCreate()

spark

In [90]:
%%sql
CREATE DATABASE IF NOT EXISTS icebergdb

++
||
++
++

# Sample Data

In [91]:
# Define the records
records = [
    (1, 'John', 25, 'NYC', '2023-09-28 00:00:00'),
    (2, 'Emily', 30, 'SFO', '2023-09-28 00:00:00'),
    (3, 'Michael', 35, 'ORD', '2023-09-28 00:00:00'),
    (4, 'Andrew', 40, 'NYC', '2023-10-28 00:00:00'),
    (5, 'Bob', 28, 'SEA', '2023-09-23 00:00:00'),
    (6, 'Charlie', 31, 'DFW', '2023-08-29 00:00:00')
]

# Define the schema
schema = StructType([
    StructField("id", IntegerType(), False),
    StructField("name", StringType(), True),
    StructField("age", IntegerType(), True),
    StructField("city", StringType(), True),
    StructField("create_ts", StringType(), True)
])

# Create a DataFrame
df = spark.createDataFrame(records, schema)
df.show()

# Create a temporary view
df.createOrReplaceTempView("temp_people")


+---+-------+---+----+-------------------+
| id|   name|age|city|          create_ts|
+---+-------+---+----+-------------------+
|  1|   John| 25| NYC|2023-09-28 00:00:00|
|  2|  Emily| 30| SFO|2023-09-28 00:00:00|
|  3|Michael| 35| ORD|2023-09-28 00:00:00|
|  4| Andrew| 40| NYC|2023-10-28 00:00:00|
|  5|    Bob| 28| SEA|2023-09-23 00:00:00|
|  6|Charlie| 31| DFW|2023-08-29 00:00:00|
+---+-------+---+----+-------------------+



# Create Table

In [92]:
%%sql
CREATE TABLE IF NOT EXISTS icebergdb.people (
    id INT,
    name STRING,
    age INT,
    city STRING,
    create_ts STRING
)
USING iceberg
PARTITIONED BY (city)

++
||
++
++

# Inserts

In [93]:
%%sql
INSERT INTO icebergdb.people
SELECT * FROM temp_people

++
||
++
++

In [94]:
%%sql
SELECT * FROM icebergdb.people order by id asc

id,name,age,city,create_ts
1,John,25,NYC,2023-09-28 00:00:00
2,Emily,30,SFO,2023-09-28 00:00:00
3,Michael,35,ORD,2023-09-28 00:00:00
4,Andrew,40,NYC,2023-10-28 00:00:00
5,Bob,28,SEA,2023-09-23 00:00:00
6,Charlie,31,DFW,2023-08-29 00:00:00


In [95]:
%%sql
SELECT * FROM icebergdb.people.history;

made_current_at,snapshot_id,parent_id,is_current_ancestor
2024-08-06 21:33:59.073000,1123456840668910264,None,True


# Delete Example

In [96]:
%%sql
DELETE FROM icebergdb.people WHERE id = 6;

++
||
++
++

In [97]:
%%sql
SELECT * FROM icebergdb.people order by id asc

id,name,age,city,create_ts
1,John,25,NYC,2023-09-28 00:00:00
2,Emily,30,SFO,2023-09-28 00:00:00
3,Michael,35,ORD,2023-09-28 00:00:00
4,Andrew,40,NYC,2023-10-28 00:00:00
5,Bob,28,SEA,2023-09-23 00:00:00


In [98]:
%%sql
SELECT * FROM icebergdb.people.history;

made_current_at,snapshot_id,parent_id,is_current_ancestor
2024-08-06 21:33:59.073000,1123456840668910264,None,True
2024-08-06 21:34:17.452000,5077088720527791611,1123456840668910264,True


# Update 

In [99]:
%%sql
MERGE INTO icebergdb.people AS target
USING (
    SELECT 
        1 AS id, 
        'John*' AS name, 
        25 AS age, 
        'NYC' AS city, 
        '2023-09-28 00:00:00' AS create_ts
) AS source
ON target.id = source.id
WHEN MATCHED THEN
    UPDATE SET 
        target.name = source.name,
        target.age = source.age,
        target.city = source.city,
        target.create_ts = source.create_ts
WHEN NOT MATCHED THEN
    INSERT (id, name, age, city, create_ts)
    VALUES (source.id, source.name, source.age, source.city, source.create_ts);

++
||
++
++

In [100]:
%%sql
SELECT * FROM icebergdb.people

id,name,age,city,create_ts
2,Emily,30,SFO,2023-09-28 00:00:00
1,John*,25,NYC,2023-09-28 00:00:00
4,Andrew,40,NYC,2023-10-28 00:00:00
5,Bob,28,SEA,2023-09-23 00:00:00
3,Michael,35,ORD,2023-09-28 00:00:00


In [101]:
%%sql
SELECT * FROM icebergdb.people.history;

made_current_at,snapshot_id,parent_id,is_current_ancestor
2024-08-06 21:33:59.073000,1123456840668910264,None,True
2024-08-06 21:34:17.452000,5077088720527791611,1123456840668910264,True
2024-08-06 21:34:20.645000,4585469957348916900,5077088720527791611,True


# Schema Evolution

In [102]:
%%sql
ALTER TABLE icebergdb.people RENAME COLUMN name TO customer_name

++
||
++
++

In [103]:
%%sql
SELECT * FROM icebergdb.people order by id asc

id,customer_name,age,city,create_ts
1,John*,25,NYC,2023-09-28 00:00:00
2,Emily,30,SFO,2023-09-28 00:00:00
3,Michael,35,ORD,2023-09-28 00:00:00
4,Andrew,40,NYC,2023-10-28 00:00:00
5,Bob,28,SEA,2023-09-23 00:00:00


In [104]:
%%sql
SELECT * FROM icebergdb.people.history;

made_current_at,snapshot_id,parent_id,is_current_ancestor
2024-08-06 21:33:59.073000,1123456840668910264,None,True
2024-08-06 21:34:17.452000,5077088720527791611,1123456840668910264,True
2024-08-06 21:34:20.645000,4585469957348916900,5077088720527791611,True


# Time Travel

In [105]:
%%sql --var df
SELECT * FROM icebergdb.people.history

made_current_at,snapshot_id,parent_id,is_current_ancestor
2024-08-06 21:33:59.073000,1123456840668910264,None,True
2024-08-06 21:34:17.452000,5077088720527791611,1123456840668910264,True
2024-08-06 21:34:20.645000,4585469957348916900,5077088720527791611,True


In [106]:
original_snapshot = df.head().snapshot_id
print("original_snapshot",original_snapshot)

spark.sql(f"CALL system.rollback_to_snapshot('icebergdb.people', {original_snapshot})")

original_snapshot


original_snapshot 1123456840668910264


1123456840668910264

In [107]:
%%sql
SELECT * FROM icebergdb.people order by id asc

id,customer_name,age,city,create_ts
1,John,25,NYC,2023-09-28 00:00:00
2,Emily,30,SFO,2023-09-28 00:00:00
3,Michael,35,ORD,2023-09-28 00:00:00
4,Andrew,40,NYC,2023-10-28 00:00:00
5,Bob,28,SEA,2023-09-23 00:00:00
6,Charlie,31,DFW,2023-08-29 00:00:00


## Metadata Tables

Iceberg tables contain very rich metadata that can be easily queried. For example, you can retrieve the manifest list for any snapshot, simply by querying the table's `snapshots` table.

In [109]:
%%sql

SELECT snapshot_id, manifest_list
FROM icebergdb.people.snapshots

snapshot_id,manifest_list
1123456840668910264,s3://warehouse/icebergdb/people/metadata/snap-1123456840668910264-1-eb255d2e-3de7-4c61-9198-90868925f7a0.avro
5077088720527791611,s3://warehouse/icebergdb/people/metadata/snap-5077088720527791611-1-aa95295a-1fc5-45f1-814d-95e21426ad48.avro
4585469957348916900,s3://warehouse/icebergdb/people/metadata/snap-4585469957348916900-1-01f15fe5-393f-4077-92ff-087ed94a69fc.avro


# Delete Table

In [89]:
%%sql
DROP TABLE icebergdb.people

++
||
++
++